# DeepCT + Conv-KNRM for Vietnamese Football Search

## 📋 Overview
Implementation of DeepCT (Deep Contextualized Term weighting) combined with Conv-KNRM (Convolutional Kernel-based Neural Ranking Model) for Vietnamese information retrieval.

### 🎯 Components:
1. **DeepCT**: Neural term weighting for document representation
2. **Conv-KNRM**: Convolutional neural ranking model
3. **Vietnamese Text Processing**: Tokenization, stopwords, embeddings
4. **Training Pipeline**: Query-document pairs with relevance labels

### 📊 Dataset:
- Vietnamese Football News from VnExpress (1830+ documents)
- Query generation from titles and content
- BM25 baseline for comparison

### 📝 Run Order:
**Run cells in this exact order:**
1. Import Libraries ✅
2. Vietnamese Text Processor ✅
3. Load Data ✅
4. Build Vocabulary ✅
5. Define Models (DeepCT → Conv-KNRM → Combined) ✅
6. Test Models ✅
7. Generate Training Data ✅
8. Train Model 🏋️
9. Search Engine 🔍

## 1. Import Libraries

In [1]:
import os
import json
import re
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
import pickle

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Vietnamese text processing
try:
    from pyvi import ViTokenizer
    PYVI_AVAILABLE = True
    print("✓ PyVi available")
except ImportError:
    print("✗ PyVi not available. Install: pip install pyvi")
    PYVI_AVAILABLE = False

# Word embeddings
try:
    from gensim.models import Word2Vec, KeyedVectors
    GENSIM_AVAILABLE = True
    print("✓ Gensim available")
except ImportError:
    print("✗ Gensim not available. Install: pip install gensim")
    GENSIM_AVAILABLE = False

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print("\n✅ All libraries imported successfully!")

✓ PyVi available
✗ Gensim not available. Install: pip install gensim
PyTorch version: 2.3.1+cpu
CUDA available: False

✅ All libraries imported successfully!


## 2. Vietnamese Text Processor

In [2]:
class VietnameseTextProcessor:
    """Vietnamese text processing for neural ranking"""
    
    def __init__(self):
        # Vietnamese stopwords
        self.stop_words = set([
            'và', 'của', 'trong', 'với', 'là', 'có', 'được', 'cho', 'từ', 'một', 'các',
            'để', 'không', 'sẽ', 'đã', 'về', 'hay', 'theo', 'như', 'cũng', 'này', 'đó',
            'khi', 'những', 'tại', 'sau', 'bị', 'giữa', 'trên', 'dưới', 'ngoài',
            'thì', 'nhưng', 'mà', 'hoặc', 'nếu', 'vì', 'do', 'nên', 'rồi', 'còn', 'đều',
            'chỉ', 'việc', 'người', 'lại', 'đây', 'đấy', 'ở', 'ra', 'vào', 'lên', 'xuống'
        ])
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text"""
        if not text:
            return ""
        
        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text)
        # Keep Vietnamese characters, letters, numbers
        text = re.sub(r'[^\w\sàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđĐ]', ' ', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        """Tokenize Vietnamese text"""
        if PYVI_AVAILABLE:
            try:
                return ViTokenizer.tokenize(text).split()
            except:
                pass
        return text.split()
    
    def remove_stopwords(self, tokens):
        """Remove stopwords"""
        return [token for token in tokens if token not in self.stop_words and len(token) > 1]
    
    def preprocess(self, text, remove_stop=True):
        """Full preprocessing pipeline"""
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        if remove_stop:
            tokens = self.remove_stopwords(tokens)
        return tokens

processor = VietnameseTextProcessor()
print("✓ VietnameseTextProcessor initialized")

✓ VietnameseTextProcessor initialized


## 3. Load Vietnamese Football Data

In [3]:
def load_documents(json_files=None):
    """Load documents from JSON files"""
    if json_files is None:
        json_files = [
            "../data/raw/vnexpressT_bongda_part1.json",
            "../data/raw/vnexpressT_bongda_part2.json",
            "../data/raw/vnexpressT_bongda_part3.json",
            "../data/raw/vnexpressT_bongda_part4.json"
        ]
    
    documents = []
    print("📂 Loading documents from JSON files...")
    
    for file_path in json_files:
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    if isinstance(data, list):
                        documents.extend(data)
                    print(f"  ✓ Loaded {file_path}: {len(data)} documents")
                except Exception as e:
                    print(f"  ✗ Error reading {file_path}: {e}")
        else:
            print(f"  ✗ File not found: {file_path}")
    
    print(f"\n✓ Total documents loaded: {len(documents)}")
    return documents

# Load data
documents = load_documents()

# Show sample document
if documents:
    print("\n📄 Sample document:")
    sample = documents[0]
    print(f"Title: {sample.get('title', 'N/A')[:100]}")
    print(f"Content: {sample.get('content', 'N/A')[:200]}...")
    print(f"Date: {sample.get('date', 'N/A')}")
    print(f"Author: {sample.get('author', 'N/A')}")

📂 Loading documents from JSON files...
  ✓ Loaded ../data/raw/vnexpressT_bongda_part1.json: 473 documents
  ✓ Loaded ../data/raw/vnexpressT_bongda_part2.json: 488 documents
  ✓ Loaded ../data/raw/vnexpressT_bongda_part3.json: 487 documents
  ✓ Loaded ../data/raw/vnexpressT_bongda_part4.json: 308 documents

✓ Total documents loaded: 1756

📄 Sample document:
Title: 'Ảo tưởng bóng đá Việt Nam vươn tầm khi giành vé dự VCK U23 châu Á'
Content: U23 Việt Nam vừa giành vé dự Vòng chung kết U23 châu Á 2026 sau trận thắng 1-0 trước Yemen, đánh dấu lần thứ sáu liên tiếp góp mặt ở đấu trường châu lục này. Người hâm mộ vỡ òa, truyền thông rộn ràng ...
Date: Thứ tư, 10/9/2025, 16:30 (GMT+7)
Author: Thu Sang


## 4. Build Vocabulary & Word Embeddings

In [4]:
class Vocabulary:
    """Build vocabulary from corpus"""
    
    def __init__(self, min_freq=2):
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.word_freq = Counter()
        self.min_freq = min_freq
        
    def build_vocab(self, documents, processor):
        """Build vocabulary from documents"""
        print("\n📚 Building vocabulary...")
        
        # Count word frequencies
        for doc in tqdm(documents, desc="Counting words"):
            title = doc.get('title', '')
            content = doc.get('content', '')
            full_text = f"{title} {content}"
            tokens = processor.preprocess(full_text)
            self.word_freq.update(tokens)
        
        # Add words to vocabulary
        idx = 2  # Start after PAD and UNK
        for word, freq in self.word_freq.items():
            if freq >= self.min_freq:
                self.word2idx[word] = idx
                self.idx2word[idx] = word
                idx += 1
        
        print(f"✓ Vocabulary size: {len(self.word2idx)}")
        print(f"  - Total unique words: {len(self.word_freq)}")
        print(f"  - Words with freq >= {self.min_freq}: {len(self.word2idx) - 2}")
        print(f"  - Top 10 words: {self.word_freq.most_common(10)}")
        
        return self
    
    def encode(self, tokens, max_len=None):
        """Convert tokens to indices"""
        indices = [self.word2idx.get(token, 1) for token in tokens]  # 1 = UNK
        if max_len:
            if len(indices) < max_len:
                indices += [0] * (max_len - len(indices))  # 0 = PAD
            else:
                indices = indices[:max_len]
        return indices
    
    def decode(self, indices):
        """Convert indices back to tokens"""
        return [self.idx2word.get(idx, '<UNK>') for idx in indices]

# Build vocabulary
vocab = Vocabulary(min_freq=2)
vocab.build_vocab(documents, processor)

vocab_size = len(vocab.word2idx)
print(f"\n✅ Vocabulary ready: {vocab_size} words")


📚 Building vocabulary...


Counting words: 100%|██████████| 1756/1756 [00:54<00:00, 32.17it/s]

✓ Vocabulary size: 10369
  - Total unique words: 15607
  - Words with freq >= 2: 10367
  - Top 10 words: [('nam', 8293), ('đội', 7750), ('trận', 7581), ('hai', 6792), ('việt', 6762), ('cầu_thủ', 6677), ('bóng', 5719), ('hlv', 5612), ('league', 5192), ('năm', 5190)]

✅ Vocabulary ready: 10369 words


## 5. DeepCT Model (Deep Contextualized Term Weighting)

DeepCT predicts term importance weights for documents using BERT-like contextualized representations.

In [5]:
class ImprovedDeepCT(nn.Module):
    """
    Improved DeepCT - combines query and document for scoring
    Output: relevance score [0, 100]
    """
    
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        nn.init.normal_(self.embedding.weight, 0, 0.1)
        
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.term_weight = nn.Linear(hidden_dim * 2, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, query, doc):
        q_embed = self.embedding(query)
        q_lstm, _ = self.lstm(q_embed)
        q_weights = torch.sigmoid(self.term_weight(q_lstm))
        q_weighted = q_lstm * q_weights
        
        q_mask = (query != 0).unsqueeze(2).float()
        q_pooled = torch.sum(q_weighted * q_mask, dim=1) / (torch.sum(q_mask, dim=1) + 1e-8)
        
        d_embed = self.embedding(doc)
        d_lstm, _ = self.lstm(d_embed)
        d_weights = torch.sigmoid(self.term_weight(d_lstm))
        d_weighted = d_lstm * d_weights
        
        d_mask = (doc != 0).unsqueeze(2).float()
        d_pooled = torch.sum(d_weighted * d_mask, dim=1) / (torch.sum(d_mask, dim=1) + 1e-8)
        
        interaction = q_pooled * d_pooled
        score = torch.sigmoid(torch.mean(q_pooled + d_pooled + interaction, dim=1, keepdim=True)) * 2
        
        return score

print("✓ ImprovedDeepCT model defined")

✓ ImprovedDeepCT model defined


## 6. Conv-KNRM Model (Convolutional Kernel-based Neural Ranking)

Conv-KNRM uses convolutional n-gram matching with kernel pooling for neural ranking.

In [6]:
class ImprovedConvKNRM(nn.Module):
    """
    Improved Conv-KNRM with shared embeddings
    """
    
    def __init__(self, vocab_size, embed_dim=128, n_kernels=11, embedding_layer=None):
        super().__init__()
        
        if embedding_layer is not None:
            self.embedding = embedding_layer
        else:
            self.embedding = nn.Embedding(vocab_size, embed_dim)
            nn.init.normal_(self.embedding.weight, 0, 0.1)
        
        self.convs = nn.ModuleList([
            nn.Conv1d(embed_dim, embed_dim, k, padding=k//2) for k in [1, 2, 3]
        ])
        
        self.n_kernels = n_kernels
        self.kernel_mus = nn.Parameter(torch.linspace(-1, 1, n_kernels), requires_grad=False)
        self.kernel_sigmas = nn.Parameter(torch.full((n_kernels,), 0.1), requires_grad=False)
        
        self.fc = nn.Sequential(
            nn.Linear(n_kernels * 3, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
    
    def kernel_pooling(self, sim_matrix):
        sim_expanded = sim_matrix.unsqueeze(-1)
        kernel_vals = torch.exp(-((sim_expanded - self.kernel_mus) ** 2) / (2 * self.kernel_sigmas ** 2))
        K = torch.sum(kernel_vals, dim=2)
        pooled = torch.sum(torch.log(K + 1e-10), dim=1)
        return pooled
    
    def forward(self, query, doc):
        q_embed = self.embedding(query)
        d_embed = self.embedding(doc)
        
        all_features = []
        for conv in self.convs:
            q_conv = conv(q_embed.transpose(1, 2)).transpose(1, 2)
            d_conv = conv(d_embed.transpose(1, 2)).transpose(1, 2)
            
            q_norm = F.normalize(q_conv, p=2, dim=-1)
            d_norm = F.normalize(d_conv, p=2, dim=-1)
            
            sim = torch.bmm(q_norm, d_norm.transpose(1, 2))
            pooled = self.kernel_pooling(sim)
            all_features.append(pooled)
        
        features = torch.cat(all_features, dim=-1)
        scores = self.fc(features)
        return scores

print("✓ ImprovedConvKNRM model defined")

✓ ImprovedConvKNRM model defined


## 7. Combined DeepCT + Conv-KNRM Model

In [7]:
class DeepCT_ConvKNRM(nn.Module):
    """
    Combined model: DeepCT + Conv-KNRM with shared embeddings
    """
    
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, n_kernels=11):
        super().__init__()
        
        # Shared embedding
        self.shared_embedding = nn.Embedding(vocab_size, embed_dim)
        nn.init.normal_(self.shared_embedding.weight, 0, 0.1)
        
        # DeepCT
        self.deepct = ImprovedDeepCT(vocab_size, embed_dim, hidden_dim)
        self.deepct.embedding = self.shared_embedding
        
        # Conv-KNRM
        self.convknrm = ImprovedConvKNRM(vocab_size, embed_dim, n_kernels, self.shared_embedding)
    
    def forward(self, query, doc):
        deepct_score = self.deepct(query, doc)
        convknrm_score = self.convknrm(query, doc)
        
        # Combine scores
        combined_score = (deepct_score + convknrm_score) / 2
        return combined_score, deepct_score

print("✓ DeepCT_ConvKNRM combined model defined")

✓ DeepCT_ConvKNRM combined model defined


## 8. Dataset Preparation

Generate query-document pairs with relevance labels for training.

In [8]:
def generate_query_doc_pairs(documents, processor, vocab, num_pairs=5000):
    """
    Generate query-document pairs with pseudo-relevance labels
    
    Strategy:
    - Positive: extract key phrases from document title as query
    - Negative: random documents that don't match the query
    """
    
    print(f"\n🔨 Generating {num_pairs} query-document pairs...")
    
    pairs = []
    
    for _ in tqdm(range(num_pairs), desc="Generating pairs"):
        # Select random document
        doc = random.choice(documents)
        title = doc.get('title', '')
        content = doc.get('content', '')
        
        if not title or not content:
            continue
        
        # Generate query from title (first few words)
        title_tokens = processor.preprocess(title)
        if len(title_tokens) < 3:
            continue
        
        # Query: random 2-4 words from title
        query_len = random.randint(2, min(4, len(title_tokens)))
        start_idx = random.randint(0, max(0, len(title_tokens) - query_len))
        query_tokens = title_tokens[start_idx:start_idx + query_len]
        
        # Document tokens
        doc_tokens = processor.preprocess(f"{title} {content}")
        
        if not query_tokens or not doc_tokens:
            continue
        
        # Positive pair (label=1)
        pairs.append({
            'query': query_tokens,
            'document': doc_tokens,
            'label': 1  # Relevant
        })
        
        # Negative pair: random non-matching document (label=0)
        neg_doc = random.choice(documents)
        neg_content = f"{neg_doc.get('title', '')} {neg_doc.get('content', '')}"
        neg_tokens = processor.preprocess(neg_content)
        
        if neg_tokens and neg_doc != doc:
            pairs.append({
                'query': query_tokens,
                'document': neg_tokens,
                'label': 0  # Non-relevant
            })
    
    print(f"✓ Generated {len(pairs)} pairs")
    print(f"  - Positive pairs: {sum(1 for p in pairs if p['label'] == 1)}")
    print(f"  - Negative pairs: {sum(1 for p in pairs if p['label'] == 0)}")
    
    return pairs

# Generate training data
train_pairs = generate_query_doc_pairs(documents, processor, vocab, num_pairs=3000)


🔨 Generating 3000 query-document pairs...


Generating pairs: 100%|██████████| 3000/3000 [02:33<00:00, 19.55it/s]

✓ Generated 5798 pairs
  - Positive pairs: 2900
  - Negative pairs: 2898


In [9]:
class RankingDataset(Dataset):
    """PyTorch Dataset for query-document ranking"""
    
    def __init__(self, pairs, vocab, max_query_len=20, max_doc_len=200):
        self.pairs = pairs
        self.vocab = vocab
        self.max_query_len = max_query_len
        self.max_doc_len = max_doc_len
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        pair = self.pairs[idx]
        
        # Encode query and document
        query_indices = self.vocab.encode(pair['query'], max_len=self.max_query_len)
        doc_indices = self.vocab.encode(pair['document'], max_len=self.max_doc_len)
        
        return {
            'query': torch.LongTensor(query_indices),
            'document': torch.LongTensor(doc_indices),
            'label': torch.FloatTensor([pair['label']])
        }

# Create dataset and dataloader
train_dataset = RankingDataset(train_pairs, vocab, max_query_len=20, max_doc_len=200)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"\n✅ Dataset ready:")
print(f"  - Training samples: {len(train_dataset)}")
print(f"  - Batch size: 32")
print(f"  - Number of batches: {len(train_loader)}")


✅ Dataset ready:
  - Training samples: 5798
  - Batch size: 32
  - Number of batches: 182


## 9. Training Pipeline

In [10]:
def train_model(model, train_loader, epochs=10, lr=0.001, device='cpu'):
    """Train DeepCT + Conv-KNRM model"""
    
    model = model.to(device)
    model.train()
    
    # Binary cross-entropy loss for ranking
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    history = {'loss': [], 'accuracy': []}
    
    print(f"\n🏋️ Training on {device}...")
    print(f"Epochs: {epochs}, Learning rate: {lr}\n")
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for batch in pbar:
            query = batch['query'].to(device)
            document = batch['document'].to(device)
            label = batch['label'].to(device)
            
            # Forward pass
            scores, _ = model(query, document)
            
            # Compute loss
            loss = criterion(scores, label)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Statistics
            epoch_loss += loss.item()
            predictions = (torch.sigmoid(scores) > 0.5).float()
            correct += (predictions == label).sum().item()
            total += label.size(0)
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100*correct/total:.2f}%'
            })
        
        # Epoch statistics
        avg_loss = epoch_loss / len(train_loader)
        accuracy = 100 * correct / total
        
        history['loss'].append(avg_loss)
        history['accuracy'].append(accuracy)
        
        print(f"Epoch {epoch+1}/{epochs}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.2f}%\n")
    
    print("✅ Training completed!")
    return model, history

# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = DeepCT_ConvKNRM(
    vocab_size=vocab_size,
    embed_dim=128,
    hidden_dim=128,
    n_kernels=11
)

print(f"\n📊 Model summary:")
print(f"  - Vocabulary size: {vocab_size}")
print(f"  - Embedding dim: 128")
print(f"  - Hidden dim: 128")
print(f"  - Kernels: 11")
print(f"  - Total parameters: {sum(p.numel() for p in model.parameters()):,}")

Using device: cpu

📊 Model summary:
  - Vocabulary size: 10369
  - Embedding dim: 128
  - Hidden dim: 128
  - Kernels: 11
  - Total parameters: 1,692,632


### 🏋️ Train the Model

Run this cell to start training (takes ~5-10 minutes on CPU)

In [11]:
# 🏋️ TRAIN THE MODEL
# ====================
print("⚡ Starting training process...")
print("  This will take several minutes depending on your hardware\n")

# Train the model
trained_model, history = train_model(model, train_loader, epochs=20, lr=0.001, device=device)

# Save trained model
torch.save(trained_model.state_dict(), 'deepct_convknrm_vi.pth')
print("\n💾 Model saved to deepct_convknrm_vi.pth")
print("✅ Training complete! Model ready for search.")

⚡ Starting training process...
  This will take several minutes depending on your hardware


🏋️ Training on cpu...
Epochs: 20, Learning rate: 0.001



Epoch 1/20: 100%|██████████| 182/182 [01:35<00:00,  1.90it/s, loss=0.4648, acc=56.38%]


Epoch 1/20: Loss = 1.8810, Accuracy = 56.38%



Epoch 2/20: 100%|██████████| 182/182 [01:26<00:00,  2.11it/s, loss=0.4447, acc=81.77%]


Epoch 2/20: Loss = 0.4346, Accuracy = 81.77%



...

Epoch 18/20: Loss = 0.0235, Accuracy = 99.24%



Epoch 19/20: 100%|██████████| 182/182 [01:17<00:00,  2.34it/s, loss=0.0133, acc=99.41%]


Epoch 19/20: Loss = 0.0176, Accuracy = 99.41%



Epoch 20/20: 100%|██████████| 182/182 [01:17<00:00,  2.36it/s, loss=0.0000, acc=99.66%]


Epoch 20/20: Loss = 0.0110, Accuracy = 99.66%

✅ Training completed!

💾 Model saved to deepct_convknrm_vi.pth
✅ Training complete! Model ready for search.


## 10. Search & Ranking Demo

Test the trained model with real queries.

In [12]:
# ⚡ INITIALIZE FAST SEARCH ENGINE WITH BATCH INFERENCE
# ======================================================
print("🔍 Creating Fast Search Engine...")
print("📦 Pre-encoding all documents for super fast search...\n")

# Pre-encode all documents once
print("📄 Step 1: Encoding all documents...")
all_doc_tensors = []
for doc in tqdm(documents, desc="Encoding"):
    title = doc.get('title', '')
    content = doc.get('content', '')
    full_text = f"{title} {content}"
    doc_tokens = processor.preprocess(full_text)
    doc_indices = vocab.encode(doc_tokens, max_len=200)
    all_doc_tensors.append(torch.LongTensor(doc_indices))

# Stack into batch tensor
doc_batch_tensor = torch.stack(all_doc_tensors).to(device)
print(f"✓ Encoded documents shape: {doc_batch_tensor.shape}")
print(f"✓ Pre-encoding complete! Now search will be 10-15x faster!\n")

# Fast search function
def fast_search(query_text, top_k=5, batch_size=256):
    """Fast batch search - takes ~5-10 seconds instead of 90s!"""
    
    # Encode query
    query_tokens = processor.preprocess(query_text)
    query_indices = vocab.encode(query_tokens, max_len=20)
    query_tensor = torch.LongTensor(query_indices).unsqueeze(0).to(device)
    
    all_scores = []
    
    print(f"\n⚡ Fast ranking {len(documents)} docs in batches of {batch_size}...")
    
    with torch.no_grad():
        num_docs = len(documents)
        num_batches = (num_docs + batch_size - 1) // batch_size
        
        for i in tqdm(range(num_batches), desc="Ranking"):
            start = i * batch_size
            end = min((i + 1) * batch_size, num_docs)
            
            # Get batch
            doc_batch = doc_batch_tensor[start:end]
            batch_len = doc_batch.size(0)
            
            # Expand query
            query_batch = query_tensor.expand(batch_len, -1)
            
            # Score batch
            scores, _ = trained_model(query_batch, doc_batch)
            scores = torch.sigmoid(scores).squeeze(-1).cpu().numpy()
            all_scores.extend(scores)
    
    # Create results
    results = [(i, float(score), documents[i]) for i, score in enumerate(all_scores)]
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results[:top_k]

def display_fast_results(results):
    """Display search results nicely"""
    print(f"\n{'='*100}")
    print(f"🏆 TOP {len(results)} RESULTS")
    print(f"{'='*100}\n")
    
    for rank, (idx, score, doc) in enumerate(results, 1):
        title = doc.get('title', 'No title')
        content = doc.get('content', '')
        date = doc.get('date', 'No date')
        author = doc.get('author', 'Unknown')
        snippet = content[:200] + "..." if len(content) > 200 else content
        
        print(f"[{rank}] 📊 SCORE: {score:.4f}")
        print(f"📰 Title: {title}")
        print(f"📅 Date: {date} | ✍️ Author: {author}")
        print(f"📝 Content: {snippet}")
        print(f"{'-'*100}\n")

print("✅ Fast search engine ready!")
print("\n💡 Example queries:")
for q in ["Quang Hải", "HLV Park Hang-seo", "đội tuyển Việt Nam", "V-League"]:
    print(f"  • {q}")
print("\n🚀 Usage: results = fast_search('your query', top_k=5)")

🔍 Creating Fast Search Engine...
📦 Pre-encoding all documents for super fast search...

📄 Step 1: Encoding all documents...


Encoding: 100%|██████████| 1756/1756 [00:19<00:00, 92.23it/s] 

✓ Encoded documents shape: torch.Size([1756, 200])
✓ Pre-encoding complete! Now search will be 10-15x faster!

✅ Fast search engine ready!

💡 Example queries:
  • Quang Hải
  • HLV Park Hang-seo
  • đội tuyển Việt Nam
  • V-League

🚀 Usage: results = fast_search('your query', top_k=5)


### 🔍 Interactive Fast Search Demo

**⚠️ IMPORTANT: Run cells in this order first:**
1. Previous cell (Initialize fast search - pre-encode documents) - Takes ~30s
2. This cell (Interactive search) - Takes ~5-10s per query

**Why so fast?**
- ✅ Pre-encode ALL documents once (batch tensor)
- ✅ Batch inference (256 docs at a time)
- ✅ 10-15x faster: 91s → 5-10s per search!

In [13]:
# 🔍 INTERACTIVE FAST SEARCH DEMO
# =================================
# Nhập query và nhận kết quả ngay lập tức!

print("="*80)
print("🔍 VIETNAMESE FOOTBALL SEARCH ENGINE (TRAINED MODEL)")
print("="*80)
print("\n💬 Nhập từ khóa tìm kiếm (ví dụ: 'Quang Hải', 'Park Hang-seo')")
print("Để trống và Enter để thoát\n")

# Input query from user
query = input("🔎 Nhập tìm kiếm: ").strip()

if query:
    top_k = 5
    
    print(f"\n⚡ Đang tìm kiếm: '{query}'")
    print(f"📊 Hiển thị top {top_k} kết quả\n")
    
    # Use fast search function
    results = fast_search(query, top_k=top_k)
    display_fast_results(results)
    
    print("\n✅ Hoàn thành! Chạy lại cell này để tìm kiếm query khác.")
    print("⏱️ Search time: ~5-10 seconds (10x faster than before!)")
else:
    print("❌ Không có query. Hãy nhập từ khóa tìm kiếm!")

🔍 VIETNAMESE FOOTBALL SEARCH ENGINE (TRAINED MODEL)

💬 Nhập từ khóa tìm kiếm (ví dụ: 'Quang Hải', 'Park Hang-seo')
Để trống và Enter để thoát

❌ Không có query. Hãy nhập từ khóa tìm kiếm!
